In [1]:
import random
import time
import threading

In [2]:
def sleep(inter_num, interval):
    for i in range(inter_num):
        print("I am alife {}".format(i))
        time.sleep(interval)
t = threading.Thread(target=sleep, args=(5, 1))
t.start()

I am alife 0
I am alife 1
I am alife 2
I am alife 3
I am alife 4


In [30]:
# TODO: давайте починим output


def sleep(inter_num , interval, lock):
    for i in range(inter_num):
        with lock:
            print("I am alife {}".format(i))
        time.sleep(interval)

inter_num = 5
interval = 1
lock = threading.Lock()
t = threading.Thread(target=sleep, args=(inter_num, interval, lock))
t.start()
#t.join()

for i in range(inter_num):
    with lock:
        print("I am boss {}".format(i))
    time.sleep(interval)

I am alife 0
I am boss 0
I am boss 1
I am alife 1
I am boss 2
I am alife 2
I am boss 3
I am alife 3
I am boss 4
I am alife 4


In [32]:
for i in range(inter_num):
    print("I am boss {}".format(i))
    time.sleep(interval)
    
for i in range(inter_num):
    print("I am oss {}".format(i))
    time.sleep(interval)

I am boss 0
I am boss 1
I am boss 2
I am boss 3
I am boss 4
I am oss 0
I am oss 1
I am oss 2
I am oss 3
I am oss 4


In [21]:
class Sleeper(threading.Thread):
    def __init__(self, name, inter_num, interval):
        super().__init__()
        self.name = name
        self.iter_num = inter_num
        self.interval = interval
    
    def run(self):
        for i in range(self.iter_num):
            print("I am alife {} my name {}".format(i, self.name))
            time.sleep(self.interval)

inter_num = 5
interval = 1
            
t = Sleeper("Ilya", inter_num, interval)
t.start()

for i in range(inter_num):
    print("I am boss {}".format(i))
    time.sleep(interval)

I am alife 0 my name Ilya
I am boss 0
I am alife 1 my name IlyaI am boss 1

I am alife 2 my name Ilya
I am boss 2
I am alife 3 my name Ilya
I am boss 3
I am boss 4
I am alife 4 my name Ilya


In [ ]:
# TODO: Создадим 6 потоков по числу преподов Vlad, Zarina, Misha, Ilya, Vlad, Anton

In [9]:
# join
class Sleeper(threading.Thread):
    def __init__(self, name, inter_num, interval):
        super().__init__()
        self.name = name
        self.iter_num = inter_num
        self.interval = interval
    
    def run(self):
        for i in range(self.iter_num):
            print("I am alife {} my name {}".format(i, self.name))
            time.sleep(self.interval)

inter_num = 5
interval = 1
            
t = Sleeper("Ilya", inter_num, interval)
t.start()
t.join()

for i in range(inter_num):
    print("I am boss {}".format(i))
    time.sleep(interval)

I am alife 0 my name Ilya
I am alife 1 my name Ilya
I am alife 2 my name Ilya
I am alife 3 my name Ilya
I am alife 4 my name Ilya
I am boss 0
I am boss 1
I am boss 2
I am boss 3
I am boss 4


In [ ]:
# TODO: Заставим преподов из задачки выше говорить последовательно

In [17]:
class DollarsAccount:
    def __init__(self, dollars_amount):
        self._dollars = dollars_amount

In [33]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.account._dollars -= self.amount
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.account._dollars += self.amount

In [19]:
dac = DollarsAccount(50)
inflation = Inflation("inflation", dac, 1, 100000)      
deposit = DepositCap("DepositCap", dac, 1, 100000)

inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

My name inflation account id 93981627515936
My name DepositCap account id 93981627515936
37133


In [36]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = threading.Lock()
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.lock.acquire()
            self.account._dollars -= self.amount
            self.lock.release()
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = threading.Lock()
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.lock.acquire()
            self.account._dollars += self.amount
            self.lock.release()

In [37]:
dac = DollarsAccount(50)
inflation = Inflation("inflation", dac, 5, 100000)      
deposit = DepositCap("DepositCap", dac, 5, 100000)

inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

#Почему не сработало?

My name inflation account id 93981627516064
My name DepositCap account id 93981627516064
108200


In [38]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num, lock):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = lock
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            with self.lock:
                self.account._dollars -= self.amount
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num, lock):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = lock
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            with self.lock:
                self.account._dollars += self.amount
            
dac = DollarsAccount(50)
lock = threading.Lock()
inflation = Inflation("inflation", dac, 5, 100000, lock)      
deposit = DepositCap("DepositCap", dac, 5, 100000, lock)

inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

My name inflation account id 93981627516064
My name DepositCap account id 93981627516064
50


In [41]:
import threading, time

a = 5
alock = threading.Lock()
b = 5
block = threading.Lock()
lock = threading.Lock()

def thread1calc():
    with lock:
        print("Thread1 acquiring lock a")
        alock.acquire()
        time.sleep(5)

        print("Thread1 acquiring lock b")
        block.acquire()
        time.sleep(5)
        a += 5
        b += 5

        print("Thread1 releasing both locks")
        block.release()
        alock.release()

def thread2calc():
    with lock:
        print ("Thread2 acquiring lock b")
        block.acquire()
        time.sleep(5)

        print("Thread2 acquiring lock a")
        alock.acquire()
        time.sleep(5)
        a += 10
        b += 10

        print("Thread2 releasing both locks")
        block.release()
        alock.release()

t1 = threading.Thread(target = thread1calc)
t1.start()

t2 = threading.Thread(target = thread2calc)
t2.start()

t1.join()
t2.join()

Thread1 acquiring lock a
Thread1 acquiring lock b


Exception in thread Thread-37:
Traceback (most recent call last):
  File "/home/verbarius/Downloads/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/verbarius/Downloads/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-41-02190af20d4a>", line 18, in thread1calc
    a += 5
UnboundLocalError: local variable 'a' referenced before assignment



Thread2 acquiring lock b


KeyboardInterrupt: 